In [ ]:
from functools import reduce, partial
from itertools import repeat
from datetime import date
import sys
import multiprocessing
import pickle

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from tqdm.notebook import tqdm
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

sys.path.append("..")

In [ ]:
# home-grown modules
import py_utils.utils as utils
import py_utils.plotting as plot_utils

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
plt.style.use("seaborn-whitegrid")
sns.set_style("whitegrid")

In [ ]:
plt.style.use("presentation.mplstyle")

In [ ]:
dict_variables = dict({
    "condition" : ["smooth", "rough"],
    "prior_sd" : [.1, 1],
    "sampling" : ["metropolis-hastings", "improvement"],
    "constrain_space" : [True, False],
    "space_edge_min" : [0],
    "space_edge_max" : [12],
    "n_features" : [2],
    "n_training" : [int(12**2/4)],
    "n_runs" : [1000]
})

In [ ]:
df_info, l_info = utils.simulation_conditions(dict_variables)

# Visualize Conditions

In [ ]:
plot_utils.plot_heatmaps(l_info)

In [ ]:
plt.style.use("presentation.mplstyle")
plot_utils.plot_1d_waves(l_info)

# Sample Training Stimuli

In [ ]:
l_df_xy = list(map(utils.make_stimuli, l_info))

In [ ]:
df_info

In [ ]:
fig, axes = plt.subplots(nrows=2,ncols=2,figsize=(20, 16), sharex="col", sharey="col",
                  gridspec_kw={"height_ratios":[1, 1]})
axes_flat = axes.flatten()
l_df_xy[0], l_ivs, scaler = utils.scale_ivs(l_df_xy[0])
l_df_xy[8], l_ivs, scaler = utils.scale_ivs(l_df_xy[8])
df_train1, df_test1 = utils.split_train_test(l_info[0], l_df_xy[0])
df_train2, df_test2 = utils.split_train_test(l_info[8], l_df_xy[8])
plot_utils.uncertainty_on_test_data(df_train1, df_test1, l_ivs, axes_flat[0:2], show_colorbar=False) # smooth
plot_utils.uncertainty_on_test_data(df_train2, df_test2, l_ivs, axes_flat[2:4], show_colorbar=False) # rough
norm = plt.Normalize(
    min(df_test1["y_pred_sd"].min(), df_test2["y_pred_sd"].min()), 
    max(df_test1["y_pred_sd"].max(), df_test2["y_pred_sd"].max())
)
sm = plt.cm.ScalarMappable(cmap="viridis", norm=norm)
sm.set_array([])
axins = inset_axes(axes_flat[3],
                    width="100%",  
                    height="5%",
                    loc='lower center',
                    borderpad=-7
                   )
_ = fig.colorbar(sm, cax=axins, orientation="horizontal")
l_df_xy[0].drop([f"""{iv}_z""" for iv in l_ivs], axis=1, inplace=True)
l_df_xy[8].drop([f"""{iv}_z""" for iv in l_ivs], axis=1, inplace=True)

*TO CONSIDERs*
- proportion of datapoints shown to participants during training (currently .25)

Metropolis-Hastings Implementation
1. compare the deviation on the current trial to the distribution of expected deviations from y_pred_sd
2. uniformly sample a value between 0 and 1
3. accept the sample if that value is smaller than the ratio in 1.

In [ ]:
n_cpus = min(len(l_info), multiprocessing.cpu_count() - 2)
p = multiprocessing.Pool(n_cpus)
#f_partial = partial(utils.run_perception, l_df_xy=l_df_xy)
# multiprocessing Pool map does not accept partial functions
list_dfs_new = []
for x in tqdm(p.starmap(utils.run_perception, zip(l_info, l_df_xy))):
    list_dfs_new.append(x)

In [ ]:
df_info

# Plot Deviation from Posterior Mean to True Values

In [ ]:
stim_id = 133 # but then 132 shows opposite pattern
l_idxs = [2, 6, 10, 14]
#l_idxs = [0, 4, 8, 12]
#l_idxs = [i + 1 for i in l_idxs]
n_plots = len(l_idxs)
n_cols = 2
n_rows = int(np.ceil(n_plots / n_cols))
f, axes = plt.subplots(n_cols, n_rows, figsize=(n_rows*4, 8), sharex="col", sharey="col")
plot_utils.plot_gp_deviations(axes, l_idxs, list_dfs_new, df_info) # , stim_id
plt.suptitle("Model Deviation from True Points after Testing Period", size=20)
plt.tight_layout()

# Save Simulated Data

In [ ]:
str_today = str(date.today())

with open(f"""../data/{str_today}-dont-refit-gp.pickle""", "wb") as f:
    pickle.dump(list_dfs_new, f)

In [ ]:
with open("../data/2021-11-14-refit-gp-1000-samples.pickle", "rb") as f:
    list_dfs_new = pickle.load(f)

*TODOs*
- save number of iteration when a sample was accepted (maybe, after a while samples are not accepted anymore because the space is populated densely)
- think about leaving in prior points when plotting movements (prior & likelihood...) as I did that in the category learning part

# Plot Movements in Space

In [ ]:
df_info

In [ ]:
f_partial = partial(plot_utils.plot_moves_one_condition, list_dfs_new=list_dfs_new, df_info=df_info)
l_idxs = df_info.index[4:8].to_list()
f, axes = plt.subplots(1, 4, figsize=(30, 8))
_ = list(map(f_partial, l_idxs, axes.flatten()))
plt.tight_layout()
plt.savefig(f"""../figures/{str_today}-func-learning-smooth-movements.png""")

In [ ]:
f_partial = partial(plot_utils.plot_moves_one_condition, list_dfs_new=list_dfs_new, df_info=df_info)
l_idxs = df_info.index[12:16].to_list()
f, axes = plt.subplots(1, 4, figsize=(30, 8))
_ = list(map(f_partial, l_idxs, axes.flatten()))
plt.tight_layout()
plt.savefig(f"""../figures/{str_today}-func-learning-rough-movements.png""")

# Correlations Between Movements in Different Conditions

In [ ]:
l_df_movements = list(map(utils.add_angle_of_movements, list_dfs_new))

In [ ]:
def correlation_between_movement_angles(idx1: int, idx2:int, l_df_movements: list) -> float:
    df_tmp = pd.merge(l_df_movements[idx1], l_df_movements[idx2], how="inner", left_index=True, right_index=True)
    return np.corrcoef(df_tmp["angle_x"], df_tmp["angle_y"])[0, 1]

In [ ]:
df1 = pd.Series(range(len(l_df_movements)), name="Condition 1")
df2 = pd.Series(range(len(l_df_movements)), name="Condition 2")
df_cross = pd.merge(df1, df2, how="cross")
f_partial = partial(correlation_between_movement_angles, l_df_movements = l_df_movements)
df_cross["correlation"] = list(map(f_partial, df_cross["Condition 1"].to_list(), df_cross["Condition 2"].to_list()))

In [ ]:
filt_info = (df_info["prior_sd"] == 1)# & (df_info["constrain_space"] == False)
idxs_required = df_info.loc[filt_info].index.to_list()
filt = (df_cross["Condition 1"].isin(idxs_required)) & (df_cross["Condition 2"].isin(idxs_required))
df_cross = df_cross.loc[filt, ].copy()

In [ ]:
df_cross["corr_above_0"] = df_cross["correlation"] > 0
prop_above_0 = (
    df_cross.query("`Condition 1` != `Condition 2`")["corr_above_0"].sum() / 
    df_cross.query("`Condition 1` != `Condition 2`").shape[0]
).round(2)
print("Proportion of Correlations > 0 between Movement Directions in Simulation Conditions: ", prop_above_0)

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(10, 8))
sns.heatmap(
    df_cross.pivot(index="Condition 1", columns="Condition 2", values="correlation").round(2), 
    annot=True, annot_kws={"size":15}, linewidths=1, cmap="vlag", vmin = -1, vmax = 1, ax=ax
)
_ = ax.set_title("Pairwise Correlations Between Movement Directions in Simulation Conditions")
plt.tight_layout()
plt.savefig(f"""../figures/{str_today}-func-learning-pairwise-correlations.png""")